# Project Starter - Simple Baseline Logistic and Decision Tree Models



The project will include following tasks:
- Load dataset
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Test/Validation
- Add engineered features
- Train and tune ML model
- Provide final metrics using Validation dataset

 As part of your deliverables, I will be create scoring function. The scoring function will perform following:
- Accept dataset in the same format as provided with the project, minus "MIS_Status" column
- Load trained model and any encoders that are needed to transform data
- Transform dataset into format that can be scored with the trained model
- Score the dataset and return the results, for each record
    - Record ID
    - Record label as determined by final model (0 or 1)
    - If the model returns probabilities, assign label based on maximum F1 threshold


Deliverables:
- Jupyter notebook with complete code to manipulate data, train and tune final model
- Model and any potential encoders in the "pkl" format
- Scoring function that will load final model and encoders


The notebook includes explanations about the code and is designed to be easily followed and results replicated.

## Dataset description

The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


## Preparation


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 1500)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score

from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy

In [ ]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

### Load data

In [ ]:
data = pd.read_csv('SBA_loans_project_1.zip')

In [ ]:
print("Data shape:", data.shape)

Data shape: (809247, 20)


### I will load the data for each model seperately as I may use different feature engineering and scalers for different models.

**Review dataset**

In [ ]:
desc_df = describe_more(data)
desc_df

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,31320,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
1,State,51,object,2,3,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
2,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
3,Bank,5716,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
4,BankState,55,object,2,3,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
5,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
6,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
7,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
8,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 932}"
9,CreateJob,234,int64,1,4,"{0: 566148, 1: 56789, 2: 52162, 3: 25945, 4: 1..."


## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

In [ ]:
def clean_data(data):
    value_to_fill={}
    for col in data.columns:
        if data[col].dtype=='object':
            value_to_fill[col]='Missing'
        else:
            value_to_fill[col]=0
    data.fillna(value=value_to_fill,inplace=True)

    Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']

    for col in Columns:
        for char in ['$',',']:
            data[col] = data[col].str.replace(char,'')
        data[col]=data[col].astype(float)

    #LowDoc and RevLineCr can only be 'Y' or 'N', i.e., yes or no.
    for col in ['LowDoc' , 'RevLineCr']:
        data[col]=data[col].agg(lambda x: 'Missing' if x not in ['N','Y'] else x)

    # NAICS code must be 6 digit code: replace smaller digits with 0.
    data['NAICS']=data['NAICS'].agg(lambda x: x if len(str(x))==6 else 0)

    # Franchise code 0 and 1 means 'NO Franchise'
    data['FranchiseCode']=data['FranchiseCode'].agg(lambda x: 1 if x==1 else x)
    
    # Convert NAICS, FranchaiseCode, Zip,'NewExist' and 'UrbanRural' into object type
    for col in ['FranchiseCode','NAICS','Zip','NewExist','UrbanRural']:
        data[col]=data[col].astype(object)

    #Convert MIS into binary
    MIS_dict={'P I F': 0, 'CHGOFF': 1, 'Missing':0}
    data['MIS_Status'].replace(MIS_dict,inplace=True)
    return(data)

# Feature Engineering

In [ ]:
def feature_engineer(data):
    ''' Creates two new features: Industry and Montly_disumbersement.
        Input: Cleaned SBA Dataframe
        Output: Dataframe with feature engineered columns
    '''
    industry_codes=['11','21','22','23','31','32','33','42','44','45','48','49','51',
                    '52','53','54','55','56','61','62','71','72','81','0']
    industry_dict={'11':'Agriculture, forestry, fishing and hunting',
        '21':'Mining, quarrying, and oil and gas extraction',
        '22':'Utilities',
        '23':'Construction',
        '31':'Manufacturing',
        '32':'Manufacturing',
        '33':'Manufacturing',
        '42':'Wholesale trade',
        '44':'Retail trade',
        '45':'Retail trade',
        '48':'Transportation and warehousing',
        '49':'Transportation and warehousing',
        '51':'Information',
        '52':'Finance and insurance',
        '53':'Real estate and rental and leasing',
        '54':'Professional, scientific, and technical services',
        '55':'Management of companies and enterprises',
        '56':'Administrative and support and waste management and remediation services',
        '61':'Educational services',
        '62':'Health care and social assistance',
        '71':'Arts, entertainment, and recreation',
        '72':'Accommodation and food services',
        '81':'Other services (except public administration) 92 Public administration',
        '0':'undefined'}
    # Function to create Industry column from NAICS columns
    def add_Industry_col(data):
        data['Industry']=data['NAICS'].astype('string')
        data['Industry']=data['Industry'].apply(lambda x: x[:2] if x[:2] in industry_codes else  x=='0' )
        data['Industry'].replace(industry_dict,inplace=True)
        return data
    
    
    # Function to create monthtly disumbersement column.
    def monthly_payment(total,terms):
        if terms==0:
            monthly=total
        else:
            monthly=total/terms
        return monthly
    
    data= add_Industry_col(data)
    data['monthly_disumb'] = data.apply(lambda x: monthly_payment(x.DisbursementGross, x.Term), axis=1)
    return data

#Create a function for binning numerical columns
def bin_numerical_cols(data, drop=True,number_of_bins=40):
    '''
    Create bins for numerical variables based on quantiles
    
    Inputs:
        data:  Cleaned and feature engineered SBA dataframe
        drop:  True or False/ True drops binned numerical columns
        number_of_bins: Maximum number of quantile bins
    
    Output: Dataframe with bin columns
    '''
    #This can also be done with if condition to determine numerical columns
    numerical_cols=['Term','NoEmp','CreateJob','RetainedJob','DisbursementGross',
                       'BalanceGross','GrAppv','SBA_Appv','monthly_disumb']
    bin_columns=[]
    binned_col=[]
    bins_for_cols={}
    for col in numerical_cols:
        bins=[np.percentile(data[col].values,i) for i in np.linspace(2.5,97.5,number_of_bins-1)]
        bins=np.concatenate([[-np.inf],bins,[np.inf]])
        bins_for_cols[col]=bins
        data[col+'_bin']=pd.cut(data[col], bins,duplicates='drop')
        data[col+'_bin']=data[col+'_bin'].astype('object')
        binned_col.append(col)
        bin_columns.append(col+'_bin')
    if drop==True:
        data.drop(columns=binned_col,inplace=True)
    return data

## Categorical variables encoding

Encoding categorical variables using the techniques below.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- (If using sklearn) Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- (If using H2O) Use H2O target encoder


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [ ]:
#Import and run the created functions on the data
data=pd.read_csv('SBA_loans_project_1.zip')
data=clean_data(data)
data=feature_engineer(data)
data=bin_numerical_cols(data,drop=False,number_of_bins=30)

X_train,X_test,y_train,y_test=train_test_split(data.drop(columns=['MIS_Status'])
                                               ,data['MIS_Status'].values.reshape(-1,1), test_size=0.2, random_state=0)
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)

In [ ]:

#Categorical encoders disctionary
column_encoders=[]
#New categorical (encoded) columns
encoded_columns=[]
#Columns to drop from ML models
columns_to_drop=[]

for col in data.drop(columns='MIS_Status').columns:
    if data[col].dtype=='object':
        if data[col].nunique() <10:
            print('Onehot encoding of {}:'.format(col))
            enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
            '''Encode Training data'''
            enc.fit(X_train[[col]])
            result = enc.transform(X_train[[col]])
            ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
            result_train=pd.DataFrame(enc.transform(X_train[[col]]),columns=ohe_columns)
            X_train = pd.concat([X_train, result_train], axis=1)
            '''Encode Test data'''
            result_test=pd.DataFrame(enc.transform(X_test[[col]]),columns=ohe_columns)
            X_test = pd.concat([X_test,result_test],axis=1)
            encoded_columns = encoded_columns + ohe_columns 
            column_encoders.append([deepcopy(enc),'ohe'])
        else:
            print('Target encoding of {}:'.format(col))
            te=TargetEncoder(smoothing=0.1,handle_unknown='value')
            '''Fit and transform Training data'''
            te.fit(X_train[col],y_train)
            new_col=col+'_trg'
            X_train[new_col]=te.transform(X_train[col])
            '''Transform Test data'''
            X_test[new_col]=te.transform(X_test[col])                         
            encoded_columns.append(new_col)
            column_encoders.append([deepcopy(te),'trg'])
            
        columns_to_drop.append(col)
        
# Drop encoded columns
X_train.drop(columns=columns_to_drop,inplace=True)
X_test.drop(columns=columns_to_drop,inplace=True)

Target encoding of City:
Target encoding of State:
Target encoding of Zip:
Target encoding of Bank:
Target encoding of BankState:
Target encoding of NAICS:
Onehot encoding of NewExist:
Target encoding of FranchiseCode:
Onehot encoding of UrbanRural:
Onehot encoding of RevLineCr:
Onehot encoding of LowDoc:
Target encoding of Industry:
Target encoding of Term_bin:
Target encoding of NoEmp_bin:
Onehot encoding of CreateJob_bin:
Target encoding of RetainedJob_bin:
Target encoding of DisbursementGross_bin:
Onehot encoding of BalanceGross_bin:
Target encoding of GrAppv_bin:
Target encoding of SBA_Appv_bin:
Target encoding of monthly_disumb_bin:


# Model Training

Depending on the model of your choice, we might need to use appropriate scaler for numerical variables.

We will train at least two types of models from the below list.
From sklearn libraries:
- Logistic regression
- SVM
- Decision Tree

From H2O libraries:
- GLM
- SVM
- Naïve Bayes Classifier

# First Model. Decision Tree with Binned numerical features

In [ ]:
for balance in ['balanced',None]:
    print('Class weight ',balance)
    parameters={'max_depth':np.linspace(11,20,10),'ccp_alpha':np.linspace(0,0.001,5)}
    dtc = DecisionTreeClassifier(criterion='gini',random_state=0,class_weight=balance)
    clf = GridSearchCV(dtc, parameters, cv=4,scoring='f1_weighted',n_jobs=-1)
    clf.fit(X_train, y_train)
    print(clf.best_params_,clf.best_score_)
    print(classification_report(y_test,clf.predict(X_test)))

Class weight  balanced
{'ccp_alpha': 0.0, 'max_depth': 20.0} 0.9309651382407252
              precision    recall  f1-score   support

           0       0.96      0.94      0.95    133561
           1       0.74      0.84      0.79     28289

    accuracy                           0.92    161850
   macro avg       0.85      0.89      0.87    161850
weighted avg       0.93      0.92      0.92    161850

Class weight  None
{'ccp_alpha': 0.0, 'max_depth': 14.0} 0.9398847975207385
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    133561
           1       0.83      0.79      0.81     28289

    accuracy                           0.93    161850
   macro avg       0.89      0.88      0.88    161850
weighted avg       0.93      0.93      0.93    161850



# Second Model. Decision Tree without Binned columns

In [ ]:
 #Load and Clean, feature engineer data
data=pd.read_csv('SBA_loans_project_1.zip')
data=clean_data(data)
data=feature_engineer(data)
#data=bin_numerical_cols(data,drop=False,number_of_bins=40)

#Split data
X_train,X_test,y_train,y_test=train_test_split(data.drop(columns=['MIS_Status'])
                                               ,data['MIS_Status'].values.reshape(-1,1), test_size=0.2, random_state=0)
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)

#Encode the categorical variables

#Categorical encoders disctionary
column_encoders=[]
#New categorical (encoded) columns
encoded_columns=[]
#Columns to drop from ML models
columns_to_drop=[]

for col in data.drop(columns='MIS_Status').columns:
    if data[col].dtype=='object':
        if data[col].nunique() <10:
            print('Onehot encoding of {}:'.format(col))
            enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
            '''Encode Training data'''
            enc.fit(X_train[[col]])
            result = enc.transform(X_train[[col]])
            ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
            result_train=pd.DataFrame(enc.transform(X_train[[col]]),columns=ohe_columns)
            X_train = pd.concat([X_train, result_train], axis=1)
            '''Encode Test data'''
            result_test=pd.DataFrame(enc.transform(X_test[[col]]),columns=ohe_columns)
            X_test = pd.concat([X_test,result_test],axis=1)
            encoded_columns = encoded_columns + ohe_columns 
            column_encoders.append([deepcopy(enc),'ohe'])
        else:
            print('Target encoding of {}:'.format(col))
            te=TargetEncoder(smoothing=0.1,handle_unknown='value')
            '''Fit and transform Training data'''
            te.fit(X_train[col],y_train)
            new_col=col+'_trg'
            X_train[new_col]=te.transform(X_train[col])
            '''Transform Test data'''
            X_test[new_col]=te.transform(X_test[col])                         
            encoded_columns.append(new_col)
            column_encoders.append([deepcopy(te),'trg'])
            
        columns_to_drop.append(col)
        
# Drop encoded columns
X_train.drop(columns=columns_to_drop,inplace=True)
X_test.drop(columns=columns_to_drop,inplace=True)

# train the model

for balance in ['balanced',None]:
    print('Class weight ',balance)
    parameters={'max_depth':np.linspace(11,20,10),'ccp_alpha':np.linspace(0,0.001,5)}
    dtc = DecisionTreeClassifier(criterion='gini',random_state=0,class_weight=balance)
    clf = GridSearchCV(dtc, parameters, cv=4,scoring='f1_weighted',n_jobs=-1)
    clf.fit(X_train, y_train)
    print(clf.best_params_,clf.best_score_)
    print(classification_report(y_test,clf.predict(X_test)))

Target encoding of City:
Target encoding of State:
Target encoding of Zip:
Target encoding of Bank:
Target encoding of BankState:
Target encoding of NAICS:
Onehot encoding of NewExist:
Target encoding of FranchiseCode:
Onehot encoding of UrbanRural:
Onehot encoding of RevLineCr:
Onehot encoding of LowDoc:
Target encoding of Industry:
Class weight  balanced
{'ccp_alpha': 0.0, 'max_depth': 20.0} 0.9289014933949413
              precision    recall  f1-score   support

           0       0.96      0.93      0.95    133561
           1       0.72      0.84      0.78     28289

    accuracy                           0.92    161850
   macro avg       0.84      0.88      0.86    161850
weighted avg       0.92      0.92      0.92    161850

Class weight  None
{'ccp_alpha': 0.0, 'max_depth': 14.0} 0.9383914224316461
              precision    recall  f1-score   support

           0       0.95      0.97      0.96    133561
           1       0.83      0.77      0.80     28289

    accuracy     

# 3. Logistic Regression

In [ ]:
# Load and manipulate date
data=pd.read_csv('SBA_loans_project_1.zip')
data=clean_data(data)
data=feature_engineer(data)
data=bin_numerical_cols(data,drop=True,number_of_bins=40)

# #Split data
X_train,X_test,y_train,y_test=train_test_split(data.drop(columns=['MIS_Status'])
                                               ,data['MIS_Status'].values.reshape(-1,1), test_size=0.2, random_state=0)
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)

#Encode the categorical variables
#Categorical encoders 
column_encoders=[]
#New categorical (encoded) columns
encoded_columns=[]
#Columns to drop from ML models
columns_to_drop=[]

for col in data.drop(columns='MIS_Status').columns:
    if data[col].dtype=='object':
        if data[col].nunique() <10:
            print('Onehot encoding of {}:'.format(col))
            enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
            '''Encode Training data'''
            enc.fit(X_train[[col]])
            result = enc.transform(X_train[[col]])
            ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
            result_train=pd.DataFrame(enc.transform(X_train[[col]]),columns=ohe_columns)
            X_train = pd.concat([X_train, result_train], axis=1)
            '''Encode Test data'''
            result_test=pd.DataFrame(enc.transform(X_test[[col]]),columns=ohe_columns)
            X_test = pd.concat([X_test,result_test],axis=1)
            encoded_columns = encoded_columns + ohe_columns 
            column_encoders.append([deepcopy(enc),'ohe'])
        else:
            print('Target encoding of {}:'.format(col))
            te=TargetEncoder(smoothing=0.1,handle_unknown='value')
            '''Fit and transform Training data'''
            te.fit(X_train[col],y_train)
            new_col=col+'_trg'
            X_train[new_col]=te.transform(X_train[col])
            '''Transform Test data'''
            X_test[new_col]=te.transform(X_test[col])                         
            encoded_columns.append(new_col)
            column_encoders.append([deepcopy(te),'trg'])
            
        columns_to_drop.append(col)
        
# Drop encoded columns
X_train.drop(columns=columns_to_drop,inplace=True)
X_test.drop(columns=columns_to_drop,inplace=True)

# Scale the data
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

print('Training in progress..')
# train the model
parameters={'C': np.linspace(0,15,25), 'penalty':('l1','l2')}
logreg =LogisticRegression(class_weight='balanced',solver='liblinear')
clf = GridSearchCV(logreg,param_grid=parameters, cv=4,scoring='f1_weighted',n_jobs=-1)
clf.fit(X_train, y_train.ravel())


Target encoding of City:
Target encoding of State:
Target encoding of Zip:
Target encoding of Bank:
Target encoding of BankState:
Target encoding of NAICS:
Onehot encoding of NewExist:
Target encoding of FranchiseCode:
Onehot encoding of UrbanRural:
Onehot encoding of RevLineCr:
Onehot encoding of LowDoc:
Target encoding of Industry:
Target encoding of Term_bin:
Target encoding of NoEmp_bin:
Target encoding of CreateJob_bin:
Target encoding of RetainedJob_bin:
Target encoding of DisbursementGross_bin:
Onehot encoding of BalanceGross_bin:
Target encoding of GrAppv_bin:
Target encoding of SBA_Appv_bin:
Target encoding of monthly_disumb_bin:
Training in progress..


GridSearchCV(cv=4,
             estimator=LogisticRegression(class_weight='balanced',
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([ 0.   ,  0.625,  1.25 ,  1.875,  2.5  ,  3.125,  3.75 ,  4.375,
        5.   ,  5.625,  6.25 ,  6.875,  7.5  ,  8.125,  8.75 ,  9.375,
       10.   , 10.625, 11.25 , 11.875, 12.5  , 13.125, 13.75 , 14.375,
       15.   ]),
                         'penalty': ('l1', 'l2')},
             scoring='f1_weighted')

In [ ]:
print(clf.best_score_,clf.best_params_)

0.8623821842716267 {'C': 0.625, 'penalty': 'l2'}


In [ ]:
LogisticRegression(class_weight='balanced',solver='liblinear',C=clf.best_params_['C'],penalty=clf.best_params_['penalty'])
logreg.fit(X_train,y_train)
y_pred=logreg.predict_proba(X_test)
threshold=np.linspace(0,1,100)
score={}
for t in threshold:
    pred_class=[1 if i>=t else 0 for i in y_pred[:,1]]
    score[t]=f1_score(y_test,pred_class)

best_threshold = max(score, key=score.get)
pred_class=[1 if i>=best_threshold else 0 for i in y_pred[:,1]]
print('Best threshold is {:.2f}.'.format(best_threshold))

print('Confusion matrix : ', confusion_matrix(y_test,pred_class))
print(classification_report(y_test,pred_class))


Best threshold is 0.69.
Confusion matrix :  [[122342  11219]
 [  8506  19783]]
              precision    recall  f1-score   support

           0       0.93      0.92      0.93    133561
           1       0.64      0.70      0.67     28289

    accuracy                           0.88    161850
   macro avg       0.79      0.81      0.80    161850
weighted avg       0.88      0.88      0.88    161850



## Train final model and save artifacts

In [ ]:
import pandas as pd
def train_model(data):
    """
    Train sample model and save artifacts
    Flow:
        - Clean dataset and do feature engineering (Transfrom)
        - Encode columns
        - Train model and encoders
    
    """
    import warnings
    warnings.filterwarnings('ignore')
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import OneHotEncoder
    from category_encoders import TargetEncoder
    from copy import deepcopy
    from sklearn.tree import DecisionTreeClassifier
    import pickle
    
    '''Transform data'''
    def clean_data(data):
        '''
        Function to Handle missing and incorrect data 

        Input: SBA dataframe given by Professor

        Output: Cleaned dataframe

        '''
        #Fill missing valuas
        value_to_fill={}
        for col in data.columns:
            if data[col].dtype=='object':
                value_to_fill[col]='Missing'
            else:
                value_to_fill[col]=0
        data.fillna(value=value_to_fill,inplace=True)

        #remove dollar sign and convert to numerical
        Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
        for col in Columns:
            for char in ['$',',']:
                data[col] = data[col].str.replace(char,'')
            data[col]=data[col].astype(float)

        #LowDoc and RevLineCr can only be 'Y' or 'N', i.e., yes or no.
        for col in ['LowDoc' , 'RevLineCr']:
            data[col]=data[col].agg(lambda x: 'Missing' if x not in ['N','Y'] else x)

        # NAICS code must be 6 digit code: replace smaller digits with 0.
        data['NAICS']=data['NAICS'].agg(lambda x: x if len(str(x))==6 else 0)

        # Franchise code 0 and 1 means 'NO Franchise'
        data['FranchiseCode']=data['FranchiseCode'].agg(lambda x: 1 if x==1 else x)

        # Convert NAICS, FranchaiseCode, Zip,'NewExist' and 'UrbanRural' into object type
        for col in ['FranchiseCode','NAICS','Zip','NewExist','UrbanRural']:
            data[col]=data[col].astype(object)
            
        #Convert MIS into binary
        MIS_dict={'P I F': 0, 'CHGOFF': 1,'Missing':0}
        data['MIS_Status'].replace(MIS_dict,inplace=True)
        return(data)

    def feature_engineer(data):
        ''' Creates two new features: Industry and Montly_disumbersement.
            Input: Cleaned SBA Dataframe
            Output: Dataframe with feature engineered columns
        '''
        industry_codes=['11','21','22','23','31','32','33','42','44','45','48','49','51',
                        '52','53','54','55','56','61','62','71','72','81','0']
        industry_dict={'11':'Agriculture, forestry, fishing and hunting',
            '21':'Mining, quarrying, and oil and gas extraction',
            '22':'Utilities',
            '23':'Construction',
            '31':'Manufacturing',
            '32':'Manufacturing',
            '33':'Manufacturing',
            '42':'Wholesale trade',
            '44':'Retail trade',
            '45':'Retail trade',
            '48':'Transportation and warehousing',
            '49':'Transportation and warehousing',
            '51':'Information',
            '52':'Finance and insurance',
            '53':'Real estate and rental and leasing',
            '54':'Professional, scientific, and technical services',
            '55':'Management of companies and enterprises',
            '56':'Administrative and support and waste management and remediation services',
            '61':'Educational services',
            '62':'Health care and social assistance',
            '71':'Arts, entertainment, and recreation',
            '72':'Accommodation and food services',
            '81':'Other services (except public administration) 92 Public administration',
            '0':'undefined'}
        # Function to create Industry column from NAICS columns
        def add_Industry_col(data):
            data['Industry']=data['NAICS'].astype('string')
            data['Industry']=data['Industry'].apply(lambda x: x[:2] if x[:2] in industry_codes else  x=='0' )
            data['Industry'].replace(industry_dict,inplace=True)
            return data


        # Function to create monthtly disumbersement column.
        def monthly_payment(total,terms):
            if terms==0:
                monthly=total
            else:
                monthly=total/terms
            return monthly
        data= add_Industry_col(data)
        data['monthly_disumb'] = data.apply(lambda x: monthly_payment(x.DisbursementGross, x.Term), axis=1)

        return data

    #Apply defined functions to clean data and create new features
    data=clean_data(data)
    data=feature_engineer(data)
    
    '''Train model and encoders'''
    y = data['MIS_Status']
    X = data.drop(columns=['MIS_Status'])
    cat_encoders = {}
    drop_columns = []
    enc_columns=[]
    for col in X.columns:
        if X[col].dtype=='object':
            if X[col].nunique() <10:
                enc = OneHotEncoder(handle_unknown='ignore')
                '''Encode Training data'''
                enc.fit(X[col].values.reshape(-1,1))
                result = enc.transform(X[col].values.reshape(-1,1)).toarray()
                ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
                result_train=pd.DataFrame(result,columns=ohe_columns)
                result_train.index=X.index
                X[ohe_columns]=result_train
                cat_encoders[col] = [deepcopy(enc),"ohe"]
                enc_columns.append(col)
            else:
                te=TargetEncoder(smoothing=0.1,handle_unknown='value')
                '''Fit and transform Training data'''
                te.fit(X[[col]],y)
                new_col=col+'_trg'
                X[new_col]=te.transform(X[col])                         
                cat_encoders[col] = [deepcopy(te),"te"]
                enc_columns.append(col)
            drop_columns.append(col)
  
    clf = DecisionTreeClassifier(criterion='gini',random_state=0, ccp_alpha=0, max_depth=14,class_weight=None)
    X=X.drop(columns=drop_columns)
    clf.fit(X, y)
    
    decision_tree_file = open("decision_tree_model.pkl", "wb")
    encoders_file = open("decision_tree_encoders.pkl", "wb")
    pickle.dump(obj=clf, file=decision_tree_file)
    pickle.dump(obj=cat_encoders, file=encoders_file)
    
    decision_tree_file.close()
    encoders_file.close()
    
    return clf


In [ ]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    import warnings
    warnings.filterwarnings('ignore')
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    
    '''Clean and feature engineer'''
    def clean_data(data):
        '''
        Function to Handle missing and incorrect data 

        Input: SBA dataframe given by Professor

        Output: Cleaned dataframe

        '''
        #Fill missing valuas
        value_to_fill={}
        for col in data.columns:
            if data[col].dtype=='object':
                value_to_fill[col]='Missing'
            else:
                value_to_fill[col]=0
        data.fillna(value=value_to_fill,inplace=True)

        #remove dollar sign and convert to numerical
        Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
        for col in Columns:
            for char in ['$',',']:
                data[col] = data[col].str.replace(char,'')
            data[col]=data[col].astype(float)

        #LowDoc and RevLineCr can only be 'Y' or 'N', i.e., yes or no.
        for col in ['LowDoc' , 'RevLineCr']:
            data[col]=data[col].agg(lambda x: 'Missing' if x not in ['N','Y'] else x)

        # NAICS code must be 6 digit code: replace smaller digits with 0.
        data['NAICS']=data['NAICS'].agg(lambda x: x if len(str(x))==6 else 0)

        # Franchise code 0 and 1 means 'NO Franchise'
        data['FranchiseCode']=data['FranchiseCode'].agg(lambda x: 1 if x==1 else x)

        # Convert NAICS, FranchaiseCode, Zip,'NewExist' and 'UrbanRural' into object type
        for col in ['FranchiseCode','NAICS','Zip','NewExist','UrbanRural']:
            data[col]=data[col].astype(object)
        
        return(data)

    def feature_engineer(data):
        ''' Creates two new features: Industry and Montly_disumbersement.
            Input: Cleaned SBA Dataframe
            Output: Dataframe with feature engineered columns
        '''
        industry_codes=['11','21','22','23','31','32','33','42','44','45','48','49','51',
                        '52','53','54','55','56','61','62','71','72','81','0']
        industry_dict={'11':'Agriculture, forestry, fishing and hunting',
            '21':'Mining, quarrying, and oil and gas extraction',
            '22':'Utilities',
            '23':'Construction',
            '31':'Manufacturing',
            '32':'Manufacturing',
            '33':'Manufacturing',
            '42':'Wholesale trade',
            '44':'Retail trade',
            '45':'Retail trade',
            '48':'Transportation and warehousing',
            '49':'Transportation and warehousing',
            '51':'Information',
            '52':'Finance and insurance',
            '53':'Real estate and rental and leasing',
            '54':'Professional, scientific, and technical services',
            '55':'Management of companies and enterprises',
            '56':'Administrative and support and waste management and remediation services',
            '61':'Educational services',
            '62':'Health care and social assistance',
            '71':'Arts, entertainment, and recreation',
            '72':'Accommodation and food services',
            '81':'Other services (except public administration) 92 Public administration',
            '0':'undefined'}
        # Function to create Industry column from NAICS columns
        def add_Industry_col(data):
            data['Industry']=data['NAICS'].astype('string')
            data['Industry']=data['Industry'].apply(lambda x: x[:2] if x[:2] in industry_codes else  x=='0' )
            data['Industry'].replace(industry_dict,inplace=True)
            return data


        # Function to create monthtly disumbersement column.
        def monthly_payment(total,terms):
            if terms==0:
                monthly=total
            else:
                monthly=total/terms
            return monthly
        data= add_Industry_col(data)
        data['monthly_disumb'] = data.apply(lambda x: monthly_payment(x.DisbursementGross, x.Term), axis=1)

        return data

    #Apply defined functions to clean data and create new features
    data=clean_data(data)
    data=feature_engineer(data)
   
    X = data.copy()
    ohe_columns = ['City',
                 'State',
                 'Zip',
                 'Bank',
                 'BankState',
                 'NAICS',
                 'NewExist',
                 'FranchiseCode',
                 'UrbanRural',
                 'RevLineCr',
                 'LowDoc',
                 'Industry']
    
    '''Load Model and encoder'''
    decision_tree_file = open("decision_tree_model.pkl", "rb")
    encoders_file = open("decision_tree_encoders.pkl", "rb")
    
    clf = pickle.load(file=decision_tree_file)
    enc_dict = pickle.load(encoders_file)
    drop_columns = []
    
    '''Encode categorical columns'''
    for col in ohe_columns:
        if enc_dict[col][1]=='ohe':
            enc = enc_dict[col][0]
            result = enc.transform(X[[col]].values.reshape(-1,1)).toarray()
            ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
            result_train = pd.DataFrame(result, columns=ohe_columns)
            result_train.index=X.index
            X[ohe_columns]=result_train
        else:
            new_col=col+'_trg'
            te = enc_dict[col][0]
            X[new_col]=te.transform(X[col])   
        drop_columns.append(col)
    X=X.drop(columns=drop_columns)  
    y_pred = clf.predict(X)
    
    decision_tree_file.close()
    encoders_file.close()
    
    return y_pred

# Example use of the train and scoring function

In [ ]:
data = pd.read_csv('SBA_loans_project_1.zip')
df_1,df_2 = train_test_split(data,test_size=0.1,random_state=42)

df_3=df_2 # will be use to get actual class

df_2 = df_2.drop(columns='MIS_Status')
clf=train_model(df_1)
estimate=project_1_scoring(df_2)

from sklearn.metrics import classification_report,confusion_matrix,f1_score,roc_auc_score
target=clean_data(df_3)['MIS_Status']
print(classification_report(target,estimate))
print(confusion_matrix(target,estimate))
print('F1 score = ',f1_score(target,estimate,average='weighted'))
print('Roc-Auc score = ',roc_auc_score(target,estimate))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     66678
           1       0.83      0.77      0.80     14247

    accuracy                           0.93     80925
   macro avg       0.89      0.87      0.88     80925
weighted avg       0.93      0.93      0.93     80925

[[64395  2283]
 [ 3242 11005]]
F1 score =  0.9307817189649107
Roc-Auc score =  0.8691020710307422
